In [1]:
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# read in data
cardtext = [list(x) for x in list(np.load('data/card_texts.npy'))]
c2i = np.load('data/c2i.npy').item()
i2c = np.load('data/i2c.npy').item()

In [3]:
# test - randomize!
np.random.seed = 1337
indices = list(np.random.permutation(len(cardtext)))
cardtext = [cardtext[i] for i in indices]
# cardtext = cardtext

In [4]:
# set parameters

DROP_RATE = 0.33 # dropout
EMBEDDING_SIZE = 256 # embedding size
HIDDEN_SIZE = 256 # lstm feature vector
HIDDEN_LAYERS = 2 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 20 # context length
CARDS_PER_BATCH = 10
NUM_EPOCHS = 1000

OUT_INCREMENT = 100 # printout after n batches - and save

In [5]:
def cardGenerator(cardtext, windowsize, cards_per_batch, c2i=c2i, debug=False):
    
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]
    
    def nextcard(cardtext, idx, debug=debug):
        if debug:
            card_idx = cardtext[idx]
        else:
            card_idx = [c2i[c] for c in cardtext[idx]]
        
        return list(card_idx)
    
    # pregenerate warmup sequence
    if debug:
        sequence = list(cardtext[idx][-(windowsize):])
    else:
        sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
    i += 1
    idx = indices[i]
    for j in range(cards_per_batch):
        sequence += nextcard(cardtext, idx)
        i += 1
        idx = indices[i]

    # create matrix
    x = []
    y = []
    
    # main iterator
    while True:
        
        # generate batch (of cards_per_batch cards)
        while len(sequence) > windowsize:
            x.append(np.array(sequence[:windowsize]))
            y.append(sequence[windowsize])
            sequence.pop(0)
        
        # generate batch_size worth of window-shifted data
        # reshape for sparse_categorical_crossentropy
        sequence = []
        y = np.array(y)
        y = y[:, np.newaxis]
        # yield and reset
        yield(np.asarray(x), y)
        x, y = [], []
        
        # check for too long, reset
        if len(indices[i:]) < cards_per_batch+1:
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]
            
        # pregenerate warmup sequence
        if debug:
            sequence = list(cardtext[idx][-(windowsize):])
        else:
            sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
        i += 1
        idx = indices[i]
        for j in range(cards_per_batch):
            sequence += nextcard(cardtext, idx)
            i += 1
            idx = indices[i]

In [6]:
getbatch = cardGenerator(cardtext, WINDOW_SIZE, CARDS_PER_BATCH)

In [7]:
# predict 'Ⓔ'

def predict(startchars='none', temperature=1.0, maxlen=1000):
    
    seq_out = []
    
    if temperature=='random':
        tmp = np.random.random()
    else:
        tmp = temperature
    
    # starting sequence
    if startchars=='none':
        seq_in = [c2i['Ⓔ'] for i in range(WINDOW_SIZE)]
    
    elif startchars=='random':
        seq_in = []
        alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']
        alpha = [a for a in alpha if a in c2i.keys()]
        while len(seq_in) < WINDOW_SIZE-1:
            rnd = np.random.randint(0, len(alpha))
            seq_in += [c2i[alpha[rnd]]]
        seq_in += [c2i['Ⓔ']]
    
    else:
        s = list(startchars)
        s = s[:WINDOW_SIZE]
        seq_out =  [c2i[c] for c in s]
        while len(s) < WINDOW_SIZE:
            s.insert(0, 'Ⓔ')
        seq_in = [c2i[c] for c in s]
        
    # softmax temperature
    # scaling factor of logits = logits/temperature
    # high temp = more confident = more diverse, more mistakes
    # low temp: more conservative
    # https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras/37254117#37254117
    def sample(a, temperature=tmp):
        a = np.array(a)**(1/temperature)
        p_sum = a.sum()
        sample_temp = a/p_sum 

        # stupid fix for > 1 error
        while sum(sample_temp) > 1:
            sample_temp[0] -= 0.0001

        return np.argmax(np.random.multinomial(1, sample_temp, 1))

    for i in range(maxlen):

        # predict next char
        pred_out = model.predict(np.array(seq_in).reshape((1, WINDOW_SIZE)))
        # get index of highest pred
        idx = sample(pred_out[0])
        # save index for decoding
        seq_out.append(idx)
        # add index to input sequence
        seq_in.append(int(idx))
        # remove earliest
        seq_in.pop(0)

    # decode final sequence
    card_char = ''.join([i2c[int(i)] for i in seq_out])
    card_char = card_char.replace('Ⓔ', 'Ⓔ\n|')
    card_char = card_char.replace('·', '|')
    card_text = card_char.split('|')
    
    for f in card_text:
        print(f)
        
    return card_text

In [8]:
# set parameters

DROP_RATE = 0.33 # dropout
EMBEDDING_SIZE = 256 # embedding size
HIDDEN_SIZE = 256 # lstm feature vector
HIDDEN_LAYERS = 2 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 20 # context length
CARDS_PER_BATCH = 10
NUM_EPOCHS = 1000

OUT_INCREMENT = 100 # printout after n batches - and save

# define model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, 
                    batch_input_shape=(1, WINDOW_SIZE, )))
model.add(Dropout(DROP_RATE))
for _ in range(HIDDEN_LAYERS-1):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True, stateful=True))
model.add(LSTM(HIDDEN_SIZE, stateful=True))
model.add(Dense(VOCAB_SIZE, activation='softmax'))

In [20]:
# load model
model.load_weights('model/v2-modelweights-epoch5-cards5000.h5')

In [24]:
predict(startchars='random', temperature='random')

gouls shantson
②Ⓑ
R
sorcery
⌧
when Ⓝ enters th• permanent counters on Ⓝ.Ⓔ

stylic diners
①Ⓖ
R
creature
sliver
flying
when Ⓝ enters the battlefield, put that cards.Ⓔ

soruleas
②Ⓡ
C
creature
human
soldier
flying
whenever a copy it control.Ⓔ

dickcks padm
③Ⓑ
U
creature
dragon
flying
①Ⓑ: put an artifact tokens.
when Ⓝ enters th• prob the battlefield, if the number of creatures you control power Ⓝ deals damage to target player.
2
2Ⓔ

giant of of gron spe
②: put a cards. if you do, target artifact combat combat player gains at the beginning of your library for a player.
1
1Ⓔ

herised of of the proar

C
artifactificed control to target creature you control and as you control dies, target player life.
1
1Ⓔ

londeal delker
③Ⓦ
R
creature
elf
when Ⓝ enters the battlefield, you may change target opponent pay with only beas.
2
2Ⓔ

ampoler of stiror
②Ⓖ
C
creature
human
wolff
incer
flying
whenever this turn ratloudst, draw a card.Ⓔ

teate of ramleder
②Ⓖ
C
instant
⌧
when Ⓝ enters the battlefield, then

['gouls shantson',
 '②Ⓑ',
 'R',
 'sorcery',
 '⌧',
 'when Ⓝ enters th• permanent counters on Ⓝ.Ⓔ\n',
 'stylic diners',
 '①Ⓖ',
 'R',
 'creature',
 'sliver',
 'flying',
 'when Ⓝ enters the battlefield, put that cards.Ⓔ\n',
 'soruleas',
 '②Ⓡ',
 'C',
 'creature',
 'human',
 'soldier',
 'flying',
 'whenever a copy it control.Ⓔ\n',
 'dickcks padm',
 '③Ⓑ',
 'U',
 'creature',
 'dragon',
 'flying',
 '①Ⓑ: put an artifact tokens.',
 'when Ⓝ enters th• prob the battlefield, if the number of creatures you control power Ⓝ deals damage to target player.',
 '2',
 '2Ⓔ\n',
 'giant of of gron spe',
 '②: put a cards. if you do, target artifact combat combat player gains at the beginning of your library for a player.',
 '1',
 '1Ⓔ\n',
 'herised of of the proar',
 '',
 'C',
 'artifactificed control to target creature you control and as you control dies, target player life.',
 '1',
 '1Ⓔ\n',
 'londeal delker',
 '③Ⓦ',
 'R',
 'creature',
 'elf',
 'when Ⓝ enters the battlefield, you may change target opponent pay 